In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import preprocessing
from tqdm import tqdm
from transformers import RobertaTokenizer, TFRobertaModel
import keras
from sklearn.model_selection import KFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
seed = 23
def set_seeds(seed_value):
    tf.random.set_seed(seed_value)
    np.random.seed(seed_value)

set_seeds(seed)

In [3]:
prompts_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
summaries_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")

prompts_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv")
summaries_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv")

In [4]:
merged_test_data = summaries_test.merge(prompts_test, how= 'left', on='prompt_id')
merged_test_data[['prompt_question', 'prompt_text', 'text']] = merged_test_data[['prompt_question', 'prompt_text', 'text']].applymap(lambda x: x.replace('\r', '').replace('\n', ''))

merged_data = summaries_train.merge(prompts_train, how= 'left', on='prompt_id')
merged_data[['prompt_question', 'prompt_text', 'text']] = merged_data[['prompt_question', 'prompt_text', 'text']].applymap(lambda x: x.replace('\r', '').replace('\n', ''))

In [5]:
data = merged_data
submission_test_data = merged_test_data

In [6]:
data

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background The Third Wave experiment took plac...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background The Third Wave experiment took plac...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...


In [7]:
#PromptText ChatGPT Summarization
summary_dict = {
    "39c16e" : "The poet crafting tragedy should avoid simple plots. The change in fortune should affect a character who isn't entirely virtuous or wicked but faces downfall due to error. The best tragedy ends in misfortune. Double-thread plots are weaker, more fitting for comedy.",
    "3b9047": "Egyptian society resembled a pyramid, with gods atop, followed by pharaohs and nobles. Pharaohs held great power, responsible for protection, laws, and taxes. Viziers aided them, scribes managed records, nobles and priests held influence, soldiers maintained order, craftsmen and merchants formed the middle class, while slaves and farmers were at the bottom. Social mobility existed, allowing some to rise through education and bureaucracy.",
    "814d6b": "In 1967, history teacher Ron Jones conducted the Third Wave experiment at a California high school. Creating a disciplined movement called 'The Third Wave' he showed how people easily follow authority and adopt superiority. The experiment revealed the potential for justifying harmful actions through blind obedience.",
    "ebad26": "In a family with firsthand experience of Packingtown's meat industry, the Third Wave experiment reveals how spoiled meat is repurposed for consumption. With horrifying details, the text exposes unsanitary practices in processing, including mixing tainted meat, chemically altering odors, and using contaminated materials. The shocking narrative underscores the industry's disregard for safety."
}

In [8]:
data['chatgpt_summary'] = data['prompt_id'].map(summary_dict)
submission_test_data['chatgpt_summary'] = submission_test_data['prompt_id'].map(summary_dict)

In [9]:
data

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,chatgpt_summary
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background The Third Wave experiment took plac...,"In 1967, history teacher Ron Jones conducted t..."
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",In a family with firsthand experience of Packi...
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"Egyptian society resembled a pyramid, with god..."
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"Egyptian society resembled a pyramid, with god..."
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background The Third Wave experiment took plac...,"In 1967, history teacher Ron Jones conducted t..."
...,...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",In a family with firsthand experience of Packi...
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"Egyptian society resembled a pyramid, with god..."
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,"Egyptian society resembled a pyramid, with god..."
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,The poet crafting tragedy should avoid simple ...


In [10]:
def encoder(text):
    tokenizer = RobertaTokenizer.from_pretrained("../input/roberta-base/")
    max_len = 190
    encoded = tokenizer(
        text,
        add_special_tokens = True,
        max_length = max_len,
        padding = 'max_length',
        truncation= True,
        return_attention_mask=True
    )
    
    input_ids = np.array(encoded['input_ids'], dtype = "int32")
    attention_masks = np.array(encoded['attention_mask'], dtype = "int32")
    
    return {"input_ids": input_ids, "attention_masks": attention_masks}


In [11]:
#DEF CUSTOM LOSS?

In [12]:
def build_model():
    #units = hp.Int(name="units", min_value=10, max_value=250, step=30)
    #units2 = hp.Int(name="units2", min_value=5, max_value=250, step=30)
    #activation = hp.Choice(name="activation", values=["relu", "linear"])
    #activation2 = hp.Choice(name="activation2", values=["relu", "linear"])
    #max_len = hp.Int(name="max_len", min_value=100, max_value=200, step=10)
    max_len = 190
    model = TFRobertaModel.from_pretrained("../input/roberta-base/")
    model_ids = Input(shape=(max_len,), dtype='int32')
    model_masks = Input(shape=(max_len,), dtype='int32')
    x = model(input_ids=model_ids, 
              attention_mask=model_masks)
    x_content = tf.keras.layers.GlobalAveragePooling1D()(x.last_hidden_state)
    x_wording = tf.keras.layers.GlobalAveragePooling1D()(x.last_hidden_state) 
    content_output = Dense(1, name='content_output')(x_content)
    wording_output = Dense(1, name='wording_output')(x_wording)
    
    model = tf.keras.Model(inputs = [model_ids, model_masks], outputs = [content_output, wording_output])
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = "mse",
        metrics=["mse"])
    return model

In [13]:
X_train = encoder(list(data["text"]))
y_train = data[["content", "wording"]].values.tolist()

#y_train = []
#y_train.append(list(data[["content", "wording"]]))
#y_train.append(list(data["wording"]))

In [14]:
def scheduler(epoch):
    learning_rate = 2e-5
    if epoch <= 1:
        return learning_rate * 0.05
    else:
        return learning_rate * (0.9**epoch)
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
'''''

#Keras Auto Tuner
import keras_tuner as kt

tuner = kt.BayesianOptimization(
    build_model,
    objective = 'loss',
    max_trials = 20,
    executions_per_trial = 2,
    directory = "kt_test",
    overwrite = False
)
'''''


'\'\'\n\n#Keras Auto Tuner\nimport keras_tuner as kt\n\ntuner = kt.BayesianOptimization(\n    build_model,\n    objective = \'loss\',\n    max_trials = 20,\n    executions_per_trial = 2,\n    directory = "kt_test",\n    overwrite = False\n)\n'

In [16]:
'''''

tuner.search_space_summary()

'''''


"''\n\ntuner.search_space_summary()\n\n"

In [17]:
'''''
tuner.search(
    X_train, y_train,
    batch_size = 20,
    epochs = 12,
    validation_split=0.2,
    callbacks=[
            EarlyStopping(
                monitor='val_loss', patience=3, restore_best_weights=True
            ),
            ModelCheckpoint(
                'roberta_uspppm.h5',
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True
            ),
            callback_lr
        ],
    shuffle=True
)
'''''


"''\ntuner.search(\n    X_train, y_train,\n    batch_size = 20,\n    epochs = 12,\n    validation_split=0.2,\n    callbacks=[\n            EarlyStopping(\n                monitor='val_loss', patience=3, restore_best_weights=True\n            ),\n            ModelCheckpoint(\n                'roberta_uspppm.h5',\n                monitor='val_loss',\n                save_best_only=True,\n                save_weights_only=True\n            ),\n            callback_lr\n        ],\n    shuffle=True\n)\n"

In [18]:
'''''

top_n = 5
best_hps = tuner.get_best_hyperparameters(top_n)

for hp in best_hps:
    print("Hyperparameters:")
    for key, value in hp.values.items():
        print(f"{key}: {value}")
    print("\n")
    
'''''



'\'\'\n\ntop_n = 5\nbest_hps = tuner.get_best_hyperparameters(top_n)\n\nfor hp in best_hps:\n    print("Hyperparameters:")\n    for key, value in hp.values.items():\n        print(f"{key}: {value}")\n    print("\n")\n    \n'

In [19]:
def train_model(train_col, X_train_input_ids, X_train_attention_masks):
    model = build_model()

    model.fit(
        (np.array(X_train_input_ids), np.array(X_train_attention_masks)),
        np.array(train_col),
        epochs=12,
        shuffle=True,
        callbacks=[
            EarlyStopping(
                monitor='val_loss', patience=3, restore_best_weights=True
            ),
            ModelCheckpoint(
                'roberta_uspppm.h5',
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True
            ),
            callback_lr
        ],
        batch_size=20,
        validation_split=0.2
    )
    
    return model

In [20]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

In [21]:
#target_cols = ["content", "wording"]

best_val_metrics = float('inf')
best_model = None

for fold_index, (train_index, val_index) in enumerate(kf.split(X_train['input_ids'])):
    print(f"Training Fold {fold_index + 1}/{k}")
    
    X_train_input_ids_fold = X_train['input_ids'][train_index]
    X_train_attention_masks_fold = X_train['attention_masks'][train_index]
    
    y_train_fold = [y_train[i] for i in train_index]
    y_val_fold = [y_train[i] for i in val_index]
    
    model = train_model(y_train_fold, X_train_input_ids_fold, X_train_attention_masks_fold)
    history = model.history
    
    val_content_mse = history.history['val_content_output_mse'][-1]
    val_wording_mse = history.history['val_wording_output_mse'][-1]
    average_val_mse = (val_content_mse + val_wording_mse) / 2       
    
    if average_val_mse < best_val_metrics:
        best_val_metrics = average_val_mse
        best_model = model
        
if best_model is not None:
    best_model.save("best_model.h5")

Training Fold 1/5


Some layers from the model checkpoint at ../input/roberta-base/ were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/roberta-base/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/12
230/230 [==============================] - 249s 872ms/step - loss: 1.2151 - content_output_loss: 0.6489 - wording_output_loss: 0.5662 - content_output_mse: 0.6489 - wording_output_mse: 0.5662 - val_loss: 0.8763 - val_content_output_loss: 0.4376 - val_wording_output_loss: 0.4386 - val_content_output_mse: 0.4376 - val_wording_output_mse: 0.4386 - lr: 1.0000e-06
Epoch 2/12
230/230 [==============================] - 204s 889ms/step - loss: 0.8020 - content_output_loss: 0.4015 - wording_output_loss: 0.4005 - content_output_mse: 0.4015 - wording_output_mse: 0.4005 - val_loss: 0.7988 - val_content_output_loss: 0.3982 - val_wording_output_loss: 0.4006 - val_content_output_mse: 0.3982 - val_wording_output_mse: 0.4006 - lr: 1.0000e-06
Epoch 3/12
230/230 [==============================] - 203s 883ms/step - loss: 0.7457 - content_output_loss: 0.3745 - wording_output_loss: 0.3712 - content_output_mse: 0.3745 - wording_output_mse: 0.3712 - val_loss: 0.6460 - val_content_output_loss: 0.322

Some layers from the model checkpoint at ../input/roberta-base/ were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/roberta-base/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/12
230/230 [==============================] - 250s 891ms/step - loss: 1.4295 - content_output_loss: 0.7341 - wording_output_loss: 0.6954 - content_output_mse: 0.7341 - wording_output_mse: 0.6954 - val_loss: 0.8977 - val_content_output_loss: 0.4358 - val_wording_output_loss: 0.4619 - val_content_output_mse: 0.4358 - val_wording_output_mse: 0.4619 - lr: 1.0000e-06
Epoch 2/12
230/230 [==============================] - 204s 889ms/step - loss: 0.8284 - content_output_loss: 0.4137 - wording_output_loss: 0.4147 - content_output_mse: 0.4137 - wording_output_mse: 0.4147 - val_loss: 0.7995 - val_content_output_loss: 0.4034 - val_wording_output_loss: 0.3960 - val_content_output_mse: 0.4034 - val_wording_output_mse: 0.3960 - lr: 1.0000e-06
Epoch 3/12
230/230 [==============================] - 203s 884ms/step - loss: 0.7609 - content_output_loss: 0.3838 - wording_output_loss: 0.3771 - content_output_mse: 0.3838 - wording_output_mse: 0.3771 - val_loss: 0.6683 - val_content_output_loss: 0.335

...

Epoch 1/12
230/230 [==============================] - 256s 911ms/step - loss: 1.3558 - content_output_loss: 0.6488 - wording_output_loss: 0.7070 - content_output_mse: 0.6488 - wording_output_mse: 0.7070 - val_loss: 0.8568 - val_content_output_loss: 0.4386 - val_wording_output_loss: 0.4182 - val_content_output_mse: 0.4386 - val_wording_output_mse: 0.4182 - lr: 1.0000e-06
Epoch 2/12
230/230 [==============================] - 203s 884ms/step - loss: 0.7963 - content_output_loss: 0.4001 - wording_output_loss: 0.3962 - content_output_mse: 0.4001 - wording_output_mse: 0.3962 - val_loss: 0.7883 - val_content_output_loss: 0.3995 - val_wording_output_loss: 0.3888 - val_content_output_mse: 0.3995 - val_wording_output_mse: 0.3888 - lr: 1.0000e-06
Epoch 3/12
230/230 [==============================] - 198s 861ms/step - loss: 0.7552 - content_output_loss: 0.3771 - wording_output_loss: 0.3781 - content_output_mse: 0.3771 - wording_output_mse: 0.3781 - val_loss: 0.7060 - val_content_output_loss: 0.356

Some layers from the model checkpoint at ../input/roberta-base/ were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/roberta-base/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/12
230/230 [==============================] - 251s 893ms/step - loss: 1.4369 - content_output_loss: 0.6260 - wording_output_loss: 0.8110 - content_output_mse: 0.6260 - wording_output_mse: 0.8110 - val_loss: 0.7959 - val_content_output_loss: 0.3935 - val_wording_output_loss: 0.4024 - val_content_output_mse: 0.3935 - val_wording_output_mse: 0.4024 - lr: 1.0000e-06
Epoch 2/12
230/230 [==============================] - 204s 887ms/step - loss: 0.8045 - content_output_loss: 0.4025 - wording_output_loss: 0.4020 - content_output_mse: 0.4025 - wording_output_mse: 0.4020 - val_loss: 0.7328 - val_content_output_loss: 0.3634 - val_wording_output_loss: 0.3694 - val_content_output_mse: 0.3634 - val_wording_output_mse: 0.3694 - lr: 1.0000e-06
Epoch 3/12
230/230 [==============================] - 202s 877ms/step - loss: 0.7287 - content_output_loss: 0.3648 - wording_output_loss: 0.3638 - content_output_mse: 0.3648 - wording_output_mse: 0.3638 - val_loss: 0.8216 - val_content_output_loss: 0.407

Some layers from the model checkpoint at ../input/roberta-base/ were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/roberta-base/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/12
230/230 [==============================] - 252s 893ms/step - loss: 1.5278 - content_output_loss: 0.7175 - wording_output_loss: 0.8103 - content_output_mse: 0.7175 - wording_output_mse: 0.8103 - val_loss: 0.8932 - val_content_output_loss: 0.4364 - val_wording_output_loss: 0.4568 - val_content_output_mse: 0.4364 - val_wording_output_mse: 0.4568 - lr: 1.0000e-06
Epoch 2/12
230/230 [==============================] - 204s 889ms/step - loss: 0.8177 - content_output_loss: 0.4017 - wording_output_loss: 0.4159 - content_output_mse: 0.4017 - wording_output_mse: 0.4159 - val_loss: 0.8106 - val_content_output_loss: 0.4045 - val_wording_output_loss: 0.4061 - val_content_output_mse: 0.4045 - val_wording_output_mse: 0.4061 - lr: 1.0000e-06
Epoch 3/12
230/230 [==============================] - 201s 875ms/step - loss: 0.7510 - content_output_loss: 0.3764 - wording_output_loss: 0.3746 - content_output_mse: 0.3764 - wording_output_mse: 0.3746 - val_loss: 1.0437 - val_content_output_loss: 0.549

In [22]:
X_test = encoder(list(submission_test_data["text"]))

In [23]:
pred = best_model.predict((np.array(X_test['input_ids']), np.array(X_test['attention_masks'])))
pred

1/1 [==============================] - 3s 3s/step


[array([[-1.604908 ],
        [-1.6225681],
        [-1.5408701],
        [-1.5581026]], dtype=float32),
 array([[-1.7013865],
        [-1.7837478],
        [-1.7538958],
        [-1.7708409]], dtype=float32)]

In [24]:
sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv")

target_cols = ["content", "wording"]
for col_index, col_name in enumerate(target_cols):
    sample_submission[col_name] = pred[col_index]


In [25]:
sample_submission.to_csv("submission.csv", index = False)

In [26]:
sample_submission

,student_id,content,wording
0,000000ffffff,-1.604908,-1.701386
1,111111eeeeee,-1.622568,-1.783748
2,222222cccccc,-1.540870,-1.753896
3,333333dddddd,-1.558103,-1.770841
